# Imports

In [12]:
# Please update the default location as you deem fit
cvml_path = '/home/alex.li/git/JupiterCVML/europa/base/src/europa'

In [13]:
import json
import os
import multiprocessing as mp

import numpy as np
import pandas as pd

from cv.core.image_quality_server_side import ImageQuality
from dl.utils.config import DEFAULT_TONEMAP_PARAMS

## Datasets

In [16]:
directory = ['/data2/jupiter/datasets/20231017_halo_rgb_labeled_excluded_bad_iq', '/data/jupiter/datasets/iq_2023_v5_anno'][1]
save_path='/mnt/alex.li/sandbox1/iq_results'

side_left_tire_mask = f'{cvml_path}/cv/core/tire_masks/side_left_iq_mask.png'
side_right_tire_mask = f'{cvml_path}/cv/core/tire_masks/side_right_iq_mask.png'

# Prediction from JupiterCVML repo

In [18]:
iq = ImageQuality(num_workers=mp.cpu_count(),
                  use_progress=True,
                  side_left_tire_mask_path = side_left_tire_mask,
                  side_right_tire_mask_path = side_right_tire_mask,
                  normalization_params=DEFAULT_TONEMAP_PARAMS,
                  dataset=dataset,
                  save_path=save_path)
print(directory)
# stereo_df = pd.read_csv(os.path.join(directory, 'master_annotation_with_histogram.csv'), low_memory=False)
stereo_df = pd.read_csv(os.path.join(directory, 'annotations.csv'), low_memory=False)

/data2/jupiter/datasets/20231017_halo_rgb_labeled_excluded_bad_iq


FileNotFoundError: [Errno 2] No such file or directory: '/data2/jupiter/datasets/20231017_halo_rgb_labeled_excluded_bad_iq/annotations.csv'

In [6]:
labeled = iq.from_df(stereo_df, directory, use_progress=True)

reducing image quality labels


In [7]:
labeled['iq'] = labeled.image_quality.apply(lambda x: x.algorithm_output)
labeled['iq_features'] = labeled.image_quality.apply(lambda x: x.algorithm_features)
labeled['iq_features_total'] = labeled.iq_features.apply(lambda x: x['image_features']['total'])
labeled['iq_features_low'] = labeled.iq_features.apply(lambda x: x['image_features']['low'])
labeled['iq_features_mid'] = labeled.iq_features.apply(lambda x: x['image_features']['mid'])
labeled['iq_features_high'] = labeled.iq_features.apply(lambda x: x['image_features']['high'])
labeled['iq_features_depth_ratio'] = labeled.iq_features.apply(lambda x: x['image_features']['depth_ratio'])
labeled['iq_features_smudge'] = labeled.iq_features.apply(lambda x: x['image_features']['smudge'])
labeled['iq_features_smudge_reason'] = labeled.iq_features.apply(lambda x: x['image_features']['smudge_reason'])
labeled['iq_process_time'] = labeled.image_quality.apply(lambda x: x.algorithm_process_time)

if 'iq_ground_truth' in labeled:
    labeled['binary_iq'] = labeled.iq.apply(lambda x: 'iq' if x != 'good' else 'non_iq')
    labeled['binary_iq_ground_truth'] = labeled.iq_ground_truth.apply(lambda x: 'iq' if x != 'good' else 'non_iq')

output_path = f'{save_path}/{dataset}'
os.makedirs(output_path, exist_ok=True)
labeled.to_csv(f'{output_path}/iq.csv')

labeled.iq.value_counts()

smudge       1456
bright        611
dark          431
good          360
bad_depth       4
Name: iq, dtype: int64

## Accuracy Metrics
#### Valid only if ground-truth image_quality_labels exist

In [8]:
def accuracy(df, iq_type='overall'):
    if iq_type!='overall':
        sub_df = df[df.iq_ground_truth==iq_type]
    else:
        sub_df = df
    valid = np.sum(sub_df.iq == sub_df.iq_ground_truth)
    total = len(sub_df)
    acc = valid/total if total != 0 else 0
    print(f'{iq_type}\tproperly-predicted:{valid:4d} \ttotal-images:{total:4d}\tacc:{100*acc:.3f}%')

if 'iq_ground_truth' in labeled:
    accuracy(labeled, 'smudge')  
    accuracy(labeled, 'dark')  
    accuracy(labeled, 'bright')  
    accuracy(labeled, 'good') 
    accuracy(labeled, 'overall')
    valid = np.sum(labeled.binary_iq == labeled.binary_iq_ground_truth)
    total = len(labeled)
    acc = valid/total
    print(f'IqNonIq\tproperly-predicted:{valid:4d} \ttotal-images:{total:4d}\tacc:{100*acc:.3f}%')

smudge	properly-predicted:1452 	total-images:1968	acc:73.780%
dark	properly-predicted: 298 	total-images: 318	acc:93.711%
bright	properly-predicted: 555 	total-images: 567	acc:97.884%
good	properly-predicted:   0 	total-images:   9	acc:0.000%
overall	properly-predicted:2305 	total-images:2862	acc:80.538%
IqNonIq	properly-predicted:2493 	total-images:2862	acc:87.107%


## Combined scores -> Model FPs + Smudge/Bright/Dark + Dust(if available)

In [ ]:
# use the `output.csv after running model on the iq/relevant dataset
model_inference_output = f'{default_loc}/Desktop/tmp/image_quality/iq_2022_v8_anno/v083_484_local_tonemap_nonocclu_debris_birds_s45_output.csv'
df = pd.read_csv(model_inference_output)
fps = df[df.state == 'false_positive']
tns = df[df.state == 'true_negative']
assert len(fps) + len(tns) == len(df), 'Model outputs can only be FPs or TNs for IQ dataset'
print(f'Total fps {len(fps)}')

In [ ]:
# invalid labels. Add these to overall stops
invalid_gt = tn_iq_stats[tn_iq_stats.image_quality_label.isna()].id

In [ ]:
tn_iq_stats = labeled[labeled.id.isin(tns.id)]
bad_iq = np.sum(tn_iq_stats.binary_iq == tn_iq_stats.binary_iq_ground_truth) + len(invalid_gt)
print(f'Bad-IQ(smudge/bright/dark) in remaining TNs {bad_iq}')
accuracy(tn_iq_stats, 'overall')

In [ ]:
total = len(fps)+bad_iq
print(f'Model-FPs: {len(fps)}\n'
      f'Bad-IQ(smudge/bright/dark) in remaining TNs: {bad_iq}\n'
      f'Total {total} of {len(labeled)}\n'
      f'Percentage {total/len(labeled)*100:.4f}%')